In [1]:
import os
import json
import sqlite3
import pandas as pd
from typing import List, Dict, Any, Tuple, Optional

from formating import convert_json_to_insert_sql, convert_json_to_update_sql
from crud import insert_records, delete_records, fetch_records, get_tables

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [ ]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [ ]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users (INSERT)

The script should already be there in _assets

In [ ]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
fetch_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

---------------------

### Migrate Patients (INSERT)

Includes manual step to treat patients missing last_name

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

#### Data Cleansing:
* Identify records with null last name and either 
    * convert the null to the string 'null' in the generated script or
    * remove the record altogether if it doesn't have any useful payload

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory (INSERT)

In [ ]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

------------------

### Migrate Short Gynaecological History data (UPDATE)

In [ ]:
source_tablename = 'tblgynhistoryshort'
target_tablename = 'history_medical'
data_index = 7

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

---------

### Migrate Pathologies (UPDATE)

In [ ]:
source_tablename = 'tblpathologies'
target_tablename = 'history_medical'
data_index = 13

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

----------

### Migrate ObstetricsHistory (INSERT)

In [ ]:
source_tablename = 'tblobstetrichistory'
target_tablename = 'history_obstetrics'
data_index = 11 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### tblobstetrichistory_x NOT TO BE MIGRATED

### Migrate Short Pregnancies History data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory'
target_tablename = 'pregnancy_history'
data_index = 16 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### Migrate Short Pregnancies History X data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory_x'
target_tablename = 'pregnancy_history_x'
data_index = 17 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)
# delete_records(target_tablename, db_path)

Delete records with null TER

In [ ]:


conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
DELETE FROM pregnancy_history_x
WHERE pregnancy_id IN (SELECT id FROM pregnancy_history WHERE ter IS NULL)
""")
cursor.execute("""
DELETE FROM pregnancy_history
WHERE ter IS NULL
""")
conn.commit()

conn.close()

### Migrate Gyn History X data (INSERT)

DELETE 5,8 and 10 records

In [ ]:
source_tablename = 'tblgynhistory'
target_tablename = 'gyn_history'
data_index = 6 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("""
    UPDATE gyn_history
    SET comments = TRIM(REPLACE(REPLACE(comments, '\n\n', '\n'), '\r\n\r\n', '\n'))
    WHERE comments IS NOT NULL
""")
conn.commit()
conn.close()

## Update next_suggested_appointment date

In [3]:
from fetusapp.chatai.note_to_app_service import analyze_doctor_note
from loguru import logger

In [34]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
               SELECT patient_id
                    , date_of_visit
                    , next_suggested_appointment
                    , comments
               FROM gyn_history g
               JOIN patients p ON g.patient_id = p.id
               WHERE date_of_visit >= date('2024-06-13', '-1 year')
               AND g.is_active = 1
               AND p.is_active = 1
               AND g.created_by = 1
               GROUP BY patient_id
               ORDER BY 2 DESC;
               """)
visits = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

visits_df = pd.DataFrame(visits, columns=columns)
logger.info(f"Total visits to analyze: {len(visits_df)}")

2025-10-20 10:46:23.364 | INFO     | __main__:<module>:22 - Total visits to analyze: 397


In [35]:
visits_df.head(50)

,patient_id,date_of_visit,next_suggested_appointment,comments
0,1349,2024-05-28,2024-09-28,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-υ...
1,608,2024-05-09,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: \n-...
2,671,2024-04-24,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: \n-...
3,255,2024-04-24,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-υ...
4,880,2024-04-23,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-α...
5,752,2024-04-23,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-υ...
6,663,2024-04-23,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: \n-...
7,416,2024-04-23,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:\n-υ...
8,972,2024-04-15,None,-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: \n-...
9,565,2024-04-15,None,"-αν έχει ξεκινήσει επαφές, της παίρνω τεστ Παπ..."


In [33]:
from fetusapp import app  # Make sure this imports your Flask app

with app.app_context():
    for i, row in visits_df.head(50).iterrows():
        logger.info("="*80)
        logger.info(f"Analyzing comment #{i}:\n{row['comments']}...")
        doctor_note = row['comments']
        patient_id = row['patient_id']
        current_appointment_date = row['date_of_visit']
        logger.info(f"Using current appointment date: {current_appointment_date}")
        try:
            analyze_doctor_note(doctor_note, patient_id, current_appointment_date=current_appointment_date)
        except Exception as e:
            logger.error(f"Error analyzing note for patient {patient_id}: {e}")
        logger.info("="*80)

2025-10-20 10:41:28.699 | INFO     | __main__:<module>:5 - ================================================================================
2025-10-20 10:41:28.701 | INFO     | __main__:<module>:6 - Analyzing comment #0:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη:
-υπέρηχος εγο:
-4 μήνες μετά τη διακοπή του θηλασμού, θα κάνουμε υπέρηχο μαστών
-αν θέλει σπιράλ, θα της πω 200 ευρώ στο ΙΑΣΩ λόγω ΠΚΤ
-μου πλήρωσε 100 ευρώ...
2025-10-20 10:41:28.701 | INFO     | __main__:<module>:10 - Using current appointment date: 2024-05-28




> Entering new AgentExecutor chain...

Invoking: `fetch_next_suggested_app_date` with `{'patient_id': '1349'}`


None
Invoking: `calculate_appointment_date` with `{'reference_date': '2024-05-28', 'timeframe': '4 months'}`


2024-09-28
Invoking: `update_next_suggested_app_date` with `{'patient_id': '1349', 'new_date': '2024-09-28'}`
responded: The doctor's note suggests a follow-up appointment for a Pap test in "4 months" after today's visit date (2024-05-28). 

Using this timeframe, the suggested date for the appointment is calculated as follows:
- **current_appointment_date**: 2024-05-28
- **timeframe**: 4 months
- **suggested_date**: 2024-09-28

I also checked the patient's next scheduled appointment, which is **next_appointment_date**: None (no appointment scheduled).

Now, applying the decision logic:

- **RULE 1**: Since next_appointment_date is None, I will update the appointment to the suggested_date (2024-09-28).
- **ACTION**: UPDATE to 2024-09-28
- **REASON**: Patient has no

2025-10-20 10:41:39.984 | INFO     | fetusapp.chatai.note_to_app_service:analyze_doctor_note:304 - Validation passed on attempt 1
2025-10-20 10:41:39.985 | INFO     | __main__:<module>:15 - ================================================================================
2025-10-20 10:41:39.986 | INFO     | __main__:<module>:5 - ================================================================================
2025-10-20 10:41:39.987 | INFO     | __main__:<module>:6 - Analyzing comment #1:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: 
-υπέρηχος εγο: ΠΚΚ,
-της έγραψα να κάνει μαστογραφία και υπέρηχο μαστών και θα μου τα προσκομίσει:
-μου πλήρωσε 80 ευρώ για όλα μαζί...
2025-10-20 10:41:39.987 | INFO     | __main__:<module>:10 - Using current appointment date: 2024-05-09




> Entering new AgentExecutor chain...
No appointment suggestion found.

> Finished chain.


2025-10-20 10:41:41.797 | INFO     | fetusapp.chatai.note_to_app_service:analyze_doctor_note:304 - Validation passed on attempt 1
2025-10-20 10:41:41.798 | INFO     | __main__:<module>:15 - ================================================================================
2025-10-20 10:41:41.798 | INFO     | __main__:<module>:5 - ================================================================================
2025-10-20 10:41:41.799 | INFO     | __main__:<module>:6 - Analyzing comment #2:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: 
-υπέρηχος εγο: ενδομήτριο. Εξαρτήματα δεν έχει
-θα κάνει μαστογραφία και υπέρηχο μαστών και θα μου τα προσκομίσει: 
-μου πλήρωσε 70 ευρώ για όλα μαζί...
2025-10-20 10:41:41.799 | INFO     | __main__:<module>:10 - Using current appointment date: 2024-04-24


...

2025-10-20 10:45:34.655 | INFO     | fetusapp.chatai.note_to_app_service:analyze_doctor_note:304 - Validation passed on attempt 1
2025-10-20 10:45:34.656 | INFO     | __main__:<module>:15 - ================================================================================
2025-10-20 10:45:34.658 | INFO     | __main__:<module>:5 - ================================================================================
2025-10-20 10:45:34.658 | INFO     | __main__:<module>:6 - Analyzing comment #48:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: SUPER SOS βλέπε προηγούμενο ιστορικό!!!!
-υπέρηχος εγο: ΟΚΚ, 
-να κάνει υπέρηχο μαστών:
-μου πλήρωσε 80 ευρώ για όλα μαζί...
2025-10-20 10:45:34.659 | INFO     | __main__:<module>:10 - Using current appointment date: 2024-03-19




> Entering new AgentExecutor chain...

Invoking: `fetch_next_suggested_app_date` with `{'patient_id': '879'}`


NoneNo appointment suggestion found in the doctor's note.

> Finished chain.


2025-10-20 10:45:37.844 | INFO     | fetusapp.chatai.note_to_app_service:analyze_doctor_note:304 - Validation passed on attempt 1
2025-10-20 10:45:37.845 | INFO     | __main__:<module>:15 - ================================================================================
2025-10-20 10:45:37.847 | INFO     | __main__:<module>:5 - ================================================================================
2025-10-20 10:45:37.848 | INFO     | __main__:<module>:6 - Analyzing comment #49:
-της πήρα τεστ Παπ και το πήγα στον Λαΐνη: ίχνη ΗPV. Της έδωσα θεραπεία με papillocare και της είπα επανάληψη σε 4 μήνες και κολποσκόπηση. Αν χρειαστεί loop τότε, θα της πω με 1500 ευρώ στο ΙΑΣΩ
-υπέρηχος εγο: ΠΚΚ, χωρίς παθολογικά ευρήματα
-της έγραψα να κάνει μαστογραφία και υπέρηχο μαστών και θα μου τα προσκομίσει: 
-μου πλήρωσε 80 ευρώ για όλα μαζί...
2025-10-20 10:45:37.849 | INFO     | __main__:<module>:10 - Using current appointment date: 2024-03-19




> Entering new AgentExecutor chain...

Invoking: `fetch_next_suggested_app_date` with `{'patient_id': '877'}`


None
Invoking: `calculate_appointment_date` with `{'reference_date': '2024-03-19', 'timeframe': '4 months'}`


2024-07-19
Invoking: `update_next_suggested_app_date` with `{'patient_id': '877', 'new_date': '2024-07-19'}`
responded: The doctor's note suggests a follow-up appointment for a Pap test in "4 months" from today's visit date (2024-03-19). 

- **Suggested Date**: 2024-07-19 (calculated from the timeframe of 4 months)
- **Next Appointment Date**: None (the patient has no future scheduled appointment)

Since there is no next appointment scheduled, I will update the patient's appointment to the suggested date of 2024-07-19.

### Action:
- **Update to Suggested Date**: 2024-07-19
- **Reason**: Patient has no appointment, so schedule one.

Now, I will proceed to update the appointment.

Appointment updated to 2024-07-19The patient's appointment has been successfully updat

2025-10-20 10:45:44.966 | INFO     | fetusapp.chatai.note_to_app_service:analyze_doctor_note:304 - Validation passed on attempt 1
2025-10-20 10:45:44.967 | INFO     | __main__:<module>:15 - ================================================================================


### Migrate Medical Exams (INSERT)

In [ ]:
source_tablename = 'tblmedexam'
target_tablename = 'medical_exams'
data_index = 8 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
# delete_records(target_tablename, db_path)
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)

### ... to be continued

In [ ]:
get_tables(db_path)

--------------